# Confiabilidade de viga mista

## Importando bibliotecas

In [521]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
import import_ipynb
from math import sqrt
import pandas as pd

## Tabela de perfis

In [522]:
tabela = pd.read_excel('Catalogo-Tabela-Bitolas-Editado-2.xlsx')
display(tabela)

,massa_linear,d,bf,tw,tf,h,d',area,Ix,Wx,...,ly,Wy,ry,Zy,rt,lt,Mesa,Alma,Cw,u\n
0,13.0,148,100,4.3,4.9,138,118,16.6,635,85.8,...,82,16.4,2.22,25.5,2.60,1.72,10.20,27.49,4181,0.67
1,18.0,153,102,5.8,7.1,139,119,23.4,939,122.8,...,126,24.7,2.32,38.5,2.69,4.34,7.18,20.48,6683,0.69
2,22.5,152,152,5.8,6.6,139,119,29.0,1229,161.7,...,387,50.9,3.65,77.9,4.10,4.75,11.52,20.48,20417,0.88
3,24.0,160,102,6.6,10.3,139,115,31.5,1384,173.0,...,183,35.9,2.41,55.8,2.73,11.08,4.95,17.48,10206,0.69
4,29.8,157,153,6.6,9.3,138,118,38.5,1739,221.5,...,556,72.6,3.80,110.8,4.18,10.95,8.23,17.94,30277,0.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,154.2,623,229,14.0,24.9,573,541,196.5,125783,4038.0,...,4999,436.6,5.04,683.3,5.94,303.29,4.60,38.66,4456995,2.11
104,155.0,611,324,12.7,19.0,573,541,198.1,129583,4241.7,...,10783,665.6,7.38,1022.6,8.53,200.77,8.53,42.60,9436714,2.47
105,174.0,616,325,14.0,21.6,573,541,222.8,147754,4797.2,...,12374,761.5,7.45,1171.1,8.58,286.88,7.52,38.63,10915665,2.48
106,195.0,622,327,15.4,24.4,573,541,250.1,168484,5417.5,...,14240,870.9,7.55,"1341 ,0",8.66,405.29,6.70,35.14,12695302,2.49


## Definindo funções

In [523]:
def calculo_destribuicao_media(media: float, desvpad: float, area: bool = False) -> list:
    AMOSTRAS: int = 1000
    medias: list = []
    areas: list = []
    
    if (area):
        for i in range(AMOSTRAS):
            numero_aleatorio: float = np.random.rand()
            medias.append(norm.ppf(numero_aleatorio, media, desvpad))
            areas.append(np.pi * (medias[i] / 2) ** 2)
        
        return medias, areas

    else:
        for i in range(AMOSTRAS):
            numero_aleatorio: float = np.random.rand()
            medias.append(norm.ppf(numero_aleatorio, media, desvpad))
            
        return medias
        

## Médias - Espessura da laje

In [524]:
espessura_laje: float = 10 # cm 
desvpad_espessura_laje: float = 0.06 * espessura_laje # cm

espessuras_laje: list = calculo_destribuicao_media(espessura_laje, desvpad_espessura_laje)

## Médias - Limite de escoamento do perfil de aço

In [525]:
limite_escoamento_perfil: float = 250 * 1.08 # MPa
desvpad_escoamento_perfil: float = 0.08 * limite_escoamento_perfil # MPa

limites_escoamento_perfil: list = calculo_destribuicao_media(limite_escoamento_perfil, desvpad_escoamento_perfil)

## Médias - Limite de escoamento da armadura

In [526]:
limite_escoamento_armadura: float = 500 * 1.08 # MPa
desvpad_escoamento_armadura: float = 0.05 * limite_escoamento_armadura #MPa

limites_escoamento_armadura: list = calculo_destribuicao_media(limite_escoamento_armadura, desvpad_escoamento_armadura)

## Médias - Resistência do concreto à compressão

In [527]:
resistencia_concreto: float = 20 * 1.17 # MPa
desvpad_resistencia_concreto: float = 0.15 * resistencia_concreto # MPa

resistencias_concreto: list = calculo_destribuicao_media(resistencia_concreto, desvpad_resistencia_concreto)

## Médias - Diamêtro do rebite

In [528]:
diametro_rebite: float = 1.59 # cm
desvpad_diametro_rebite: float = 0.01 * diametro_rebite # cm

diametros_rebites, areas_rebites = calculo_destribuicao_media(diametro_rebite, desvpad_diametro_rebite, True)


## Médias - Resistência do rebite

In [529]:
resistencia_rebite: float = 415 * 1.06 # MPa
desvpad_resistencia_rebite: float = 0.06 * resistencia_rebite # MPa

resistencias_rebite: list = calculo_destribuicao_media(resistencia_rebite, desvpad_resistencia_rebite)

## Pré-dimensionamento

In [530]:
carga_permanente_g1: float = 7.6 # kN/m
carga_permanente_g2: float = 5 # kN/m

variacao_carga: list = [3, 12]
carga_utilizacao_q2: list = [i for i in range(variacao_carga[0], variacao_carga[1])]

carga_construcao_q1: list = [i / 5.6 for i in carga_utilizacao_q2]

# Propriedades da laje
vao_laje: float = 9 # m
espacamento_laje: float = 2.8 # m

# Carga distrubuída do projeto
for i in range(len(carga_utilizacao_q2)):
    carga_distribuida_projeto: list = [1.4 * (carga_permanente_g1 + carga_permanente_g2) + carga_construcao_q1[i] * carga_utilizacao_q2[i]]

# Momento solicitante do projeto
momento_solicitante_projeto: list = [(carga_distribuida_projeto[i] * (vao_laje ** 2)) / 8 for i in range(len(carga_distribuida_projeto))]

altura_perfil_ht: float = 45 # cm
coeficiente_ya1: float = 1.1
x: float = 10 # cm

# Tabela com áreas filtradas a partir de 450 mm
tabela_filtrada = tabela.loc[tabela['d'] >= 450]

row_tabela: int = 0

# Área mínima da seção de aço
areas_minimas: list = [[] for i in range(len(momento_solicitante_projeto))]
index_matriz: int = 0
area_minima_inicial: float = tabela_filtrada.iloc[0]['area']
for i in momento_solicitante_projeto:
    for j in range(len(limites_escoamento_perfil)):
        areas_minimas[index_matriz].append((i * 100) / (((limites_escoamento_perfil[j] / 10) / coeficiente_ya1) * ((altura_perfil_ht / 2) + espessuras_laje[j] - (x / 2)))) # cm²
        if (areas_minimas[index_matriz][j] > area_minima_inicial + 100):
            print(f'Área mínima inicial: {area_minima_inicial}')
            print(f'Área mínima calculada: {areas_minimas[index_matriz][j]}')
            row_tabela += 1
            area_minima_inicial = tabela_filtrada.iloc[row_tabela]['area']
    index_matriz += 1
    

print(f'Áreas mínimas: {areas_minimas}')
print(f'Área mínima final: {area_minima_inicial}')


Áreas mínimas: [[57.04755597467475, 64.55267446132436, 59.56362783161931, 54.84699236928617, 53.5019192603623, 54.4018804549753, 57.05869727737899, 58.78591528744306, 69.6880807891557, 62.53091129975241, 59.15200244969574, 64.50240746772548, 61.7325828689916, 62.857868788725526, 59.02464827666224, 64.22292675171269, 48.90273824610762, 54.25318159862487, 60.67071289438114, 61.51881220836484, 63.24881993790626, 67.0741894994298, 57.53768620770221, 57.78310039325228, 60.91151423556477, 54.90510265399234, 54.86722361016905, 61.53393440810338, 66.38016715958506, 61.042380784258064, 56.33933953425628, 53.38153422735389, 68.01181366106654, 63.88662375480562, 54.657822103444104, 57.695256315025055, 54.48026178440618, 51.73199188354992, 57.21100673220733, 55.68429878576082, 56.124067100023325, 56.18931377381675, 55.66036752447007, 63.531632057086725, 61.20490958333205, 54.36237244338011, 51.83691854474703, 57.63996253909276, 54.501348107176675, 66.65706413511388, 56.680486459493856, 56.70663878

## Flambagem da seção

In [531]:
# Classificação da seção quanto a flambagem
altura_perfil: float = tabela_filtrada.iloc[row_tabela]['d\''] # mm
altura_total_perfil: float = tabela_filtrada.iloc[row_tabela]['d'] # mm
altura_efetiva_perfil: float = tabela_filtrada.iloc[row_tabela]['h'] # mm
espessura_perfil: float = tabela_filtrada.iloc[row_tabela]['tw'] # mm
espessura_mesa: float = tabela_filtrada.iloc[row_tabela]['tf'] # mm
largura_mesa: float = tabela_filtrada.iloc[row_tabela]['bf'] # mm
modulo_elasticidade_perfil: float = 200000 # MPa
print(f'Altura perfil: {altura_perfil}, Altura total do perfil: {altura_total_perfil} e Espessura do perfil: {espessura_perfil}')

esbeltez_alma: float = altura_perfil / espessura_perfil

limites_flambagem_local: list = []

def calculo_limite_flambagem_local(coef: float, modulo_elasticidade_perfil: float, limite_escoamento_perfil: float) -> float:
    return coef * sqrt(modulo_elasticidade_perfil / limite_escoamento_perfil)

contador_compacto: int = 0
contador_semicompacto: int = 0

for i in range(len(limites_escoamento_perfil)):
    limite = calculo_limite_flambagem_local(3.76, modulo_elasticidade_perfil, limites_escoamento_perfil[i])
    limites_flambagem_local.append(limite)
    
    if esbeltez_alma <= limite:
        contador_compacto += 1
    elif esbeltez_alma > limite and esbeltez_alma < calculo_limite_flambagem_local(5.7, modulo_elasticidade_perfil, limites_escoamento_perfil[i]):
        contador_semicompacto += 1
    else:
        print('Erro')
        
print(f'Compactas: {contador_compacto}')
print(f'Semi-compactas: {contador_semicompacto}')
        

Altura perfil: 404, Altura total do perfil: 450 e Espessura do perfil: 7.6
Compactas: 1000
Semi-compactas: 0


## Momento resistência da viga mista

In [532]:
# Resistência à compressão do concreto

yc: float = 1.4
largura_efetiva_laje: float = 225 # cm

rcd: list = []

for i in range(len(resistencias_concreto)):
    rcd.append(0.85 * (resistencias_concreto[i] / 10) * largura_efetiva_laje * espessuras_laje[i] / yc)

print(f'Rcd: {rcd}')

# Resistência à tração do aço

rtd: list = []

for i in range(len(limites_escoamento_perfil)):
    rtd.append(area_minima_inicial * (limites_escoamento_perfil[i] / 10) / coeficiente_ya1)
    
print(f'Rtd: {rtd}')

profundidades_linha_neutra: list = []
momentos_resistente_laje_concreto: list = []

for i in range(len(rtd)):
    if rcd[i] >= rtd[i]:
        aux = rtd[i]/(rcd[i] / espessuras_laje[i])
        profundidades_linha_neutra.append(aux)
        momentos_resistente_laje_concreto.append(rtd[i] * ((altura_total_perfil / 20) + espessuras_laje[i] - aux / 2) / 100)
    else:
        cad: float = 0.5 * (rtd[i] - rcd[i])
        print(f'Cad: {cad}')
        intervalo = (limites_escoamento_perfil[i] / 10) * (largura_mesa / 10) * (espessura_mesa / 10) / coeficiente_ya1
        if cad < intervalo:
            yp: float = cad / ((limites_escoamento_perfil[i] / 10) * (largura_mesa / 10) / coeficiente_ya1)
            print(f'yp -> Cad MENOR intervalo: {yp}')
        else:
            yp: float = ((cad - intervalo) / ((limites_escoamento_perfil[i] / 10) * (espessura_perfil / 10) / coeficiente_ya1)) + (espessura_mesa / 10)
            print(f'yp -> Cad MAIOR intervalor: {yp}')
            
        area_tracionada_aco: float = area_minima_inicial - (largura_mesa / 10) * yp
        print(f'Área tracionada do aço: {area_tracionada_aco}')
        
        centro_gravidade_area_tracionada: float = ((((largura_mesa / 10) * ((espessura_mesa / 10) ** 2)) / 2) + (altura_efetiva_perfil / 10) * (espessura_perfil / 10) * ((altura_efetiva_perfil / 10) / 2 + (espessura_mesa / 10)) + (largura_mesa / 10) * ((espessura_mesa / 10) - yp) * (((espessura_mesa / 10) - yp) / 2 + (altura_efetiva_perfil / 10) + (espessura_mesa / 10))) / area_tracionada_aco
        print(f'Centro de gravidade da área tracionada: {centro_gravidade_area_tracionada}')
        
        momento_calculado: float = rcd[i] * ((altura_total_perfil / 10) - centro_gravidade_area_tracionada + espessuras_laje[i] / 2) + cad * (altura_total_perfil / 10 - centro_gravidade_area_tracionada - yp / 2)
        momentos_resistente_laje_concreto.append(momento_calculado / 100)
        print(f'Momento calculado: {momento_calculado / 100}')
        
print(f'x: {profundidades_linha_neutra}')
print(f'Momentos resistentes da laje de concreto:\n{momentos_resistente_laje_concreto}')



Rcd: [3946.2340401500883, 3548.521866846613, 3069.356253381212, 2791.8006625702474, 3237.140223349212, 2245.125879514711, 3447.687412926264, 3284.747194050053, 2368.369198573797, 3392.4103567410302, 4272.6385294653255, 3132.2716460766105, 3192.08159382781, 3506.98067613785, 3453.831947756025, 3727.4786525291725, 4157.489614590663, 2436.6206535300466, 3266.024732695626, 3381.7830595407513, 3207.6708278667707, 3304.537980998109, 3661.943453784578, 3567.786385045381, 3222.415872579957, 3123.4550306985943, 3772.1840582854647, 3484.5108456329726, 2939.232238552158, 3489.367629441076, 2398.9329011723385, 4012.735799865511, 3775.8551836227534, 3443.574935706681, 3197.1900428107156, 3155.86686513143, 2853.854507286632, 3467.447666930928, 3242.75070731449, 3384.2115917248407, 3466.9047592506554, 3099.4698698936836, 2829.949911239161, 3323.9881792841916, 3296.7863482508874, 4214.413990478399, 2925.4477426848575, 3332.0519570168954, 3385.72117694168, 2669.205730112793, 3500.0895071689924, 2678.07

## Confiabilidade do momento resistente

In [533]:
# Cálculo da probabilidade de falha

probabilidades_falhas: list = []

for i in momento_solicitante_projeto:
    print(i)
    falhas: int = 0
    for j in range(len(momentos_resistente_laje_concreto)):
        if momentos_resistente_laje_concreto[j] - i < 0:
            print(f'Momento solicitante: {i} e Momento resistente: {momentos_resistente_laje_concreto[j]}')
            falhas += 1
    probabilidades_falhas.append(falhas / len(momentos_resistente_laje_concreto))

print(f'Falhas: {falhas}')
print(f'Probabilidades de falha: {probabilidades_falhas}')
    

397.3773214285714
Momento solicitante: 397.3773214285714 e Momento resistente: 393.36325299706346
Momento solicitante: 397.3773214285714 e Momento resistente: 393.85421150077957
Momento solicitante: 397.3773214285714 e Momento resistente: 394.9801305148921
Momento solicitante: 397.3773214285714 e Momento resistente: 394.3091387463226
Momento solicitante: 397.3773214285714 e Momento resistente: 366.1645644985231
Momento solicitante: 397.3773214285714 e Momento resistente: 382.7626367558445
Momento solicitante: 397.3773214285714 e Momento resistente: 385.86121980281035
Falhas: 7
Probabilidades de falha: [0.007]
